## Creating Cohort Analysis in Python

In this tutorial well we looking at how to create a a cohort analysis from scratch in Python using the Pandas library and pivot table. This tutorial was originally created by <a href="https://medium.com/@henryfeng/know-users-behaviors-better-with-cohort-analysis-in-python-6c0dfc373963">Henry Feng</a> of amazon.

I've added some new aspects, update the code and adapted the tutorial for those that my not have previous experince in Python to make it easier to understand what's happening in each step.

In [ ]:
import numpy as np # calculation
import pandas as pd # Excel for Python
import datetime as dt # allow for date and time series manipulation
import seaborn as sns # a charting package
import matplotlib.pyplot as plt # the original Python chating package

#a `magic code` that allows ploting of charts within the notebook
%matplotlib inline

# Sets all rows to display
pd.options.display.max_rows = None

#Stops a false alarm chaining error
pd.options.mode.chained_assignment = None

In [ ]:
data = pd.read_excel('data/Online Retail.xlsx')

In [ ]:
data.head()

## Helper functions
Let me introduce an important part of Python. Reusable code. Resuable code in this case comes as a 'helper' function. These are little bespoke functions we build to handle reptitive tasks.

The function below aggrates the statstical and descriptive data about the data set we're using.

In [ ]:
def explore(x):
    divider = "*_*"
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Dataframe Makeup \n") #title for output.
    
    x.info() # Explains what the data and values the data is madeup from.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Descriptive Statistics \n\n", x.describe().round(2)) #Gives a statstical breakdown of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    
    print("Shape of dataframe: {}".format(x.shape)) # Gives the shape of the data.
    
    print("\n {} \n".format((divider*20))) #creates a dvider between each method output breaking at each end.
    return

In [ ]:
explore(data)

## Cleaning Data
In order to use data in Python you need to be explicit. What that means is remove or replace data which isn't valid. In many cases this also means transforming data but we will not cover this in this tutorial.

In this section we want to clean up the data so that there are no missing values.

In [ ]:
#Create a function that sums all of the missing data from each row so that we can count what we have

def missing_data(x):
    return x.isna().sum()

In [ ]:
missing_data(data)

In [ ]:
#drops missing data from the CustomerID column
cleaned_data = data.dropna(subset=['CustomerID'])

In [ ]:
explore(cleaned_data)

In [ ]:
cleaned_data.head() #view the top 5 rows of data

## Munging and Wrangling data

Whenever using Python you will always need to munge or wrangle the data. These are fun sound words for formating and processing data. Unlike software such as Excel or GSheets Pandas doesn't try to guess formats. You need to be explicit in the formating of each column of data.

In [ ]:
#Uses the datetime function to gets the month a datetime stamp and strips the time
def get_month(x):
    return dt.datetime(x.year, x.month, 1) #year, month, incremints of day

In [ ]:
#Create a new column 
cleaned_data['InvoiceMonth'] = cleaned_data['InvoiceDate'].apply(get_month) 

In [ ]:
#Always inspect the data you've just created
cleaned_data['InvoiceMonth']

In [ ]:
#Create a CohortMonth column by grouping data and selecting the earliest instance in the data. 
cleaned_data['CohortMonth'] = cleaned_data.groupby('CustomerID')['InvoiceMonth'].transform('min')

In [ ]:
cleaned_data['CohortMonth']

In [ ]:
cleaned_data.head()

## Create the Cohort
So far we've been doing the admin. Inspecting and cleaning up the data is a key part in any data anlysis with Python. Now we will create a cohort which to analyise and visualise...but first, you guessed it, more data processing.

Here we are going to create the cohort index. This will give the month lapsed number first and last transaction.

In [ ]:
#When passed a datetime column this functions splits out year, month, day

def get_date(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [ ]:
#splits invoiced month and data into single variables
invoice_year, invoice_month, _ = get_date(cleaned_data, 'InvoiceMonth') 

In [ ]:
#Inspect the variable
invoice_month[:30] #[:30] selects the first 30 rows of data

In [ ]:
#Inspect the variable
invoice_year[:30] #[:30] selects the first 30 rows of data

In [ ]:
#splits cohort month and data into single variables
cohort_year, cohort_month, _ = get_date(cleaned_data, 'CohortMonth')

In [ ]:
cohort_month[:30]

In [ ]:
cohort_year[:30]

In [ ]:
# Creating a variable which holds the differnce between the invoice and cohort year 
year_diff = invoice_year - cohort_year

In [ ]:
year_diff

In [ ]:
# Creating a variable which holds the differnce between the invoice and cohort month 
month_diff = invoice_month - cohort_month

In [ ]:
month_diff

In [ ]:
#Now creating a column that has the calclation shows the 
cleaned_data['CohortIndex'] = year_diff * 12 + month_diff + 1

In [ ]:
cleaned_data['CohortIndex']

In [ ]:
cleaned_data.head()


## Cohort Analysis 1

Finally we create a Cohort Analysis. We're using, what I deem to be, the top most used function for marketering doing data analysis <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">`pd.groupby()`</a>. This allows you to group data by a specific column. Then we'll be using <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html">`pd. pivot_table()`</a> to convert the data into a what you would typically recognise as a Cohort Table.

In [ ]:
#Group the data by columns CohortMonth','CohortIndex' then aggreate by column 'CustomerID'
cohort_data = cleaned_data.groupby(
    ['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()

In [ ]:
#Take the cohort_data and plumb it into a Pivot Table. Setting index, columns and values as below.
cohort_count = cohort_data.pivot_table(index = 'CohortMonth',
                                       columns = 'CohortIndex',
                                       values = 'CustomerID')

In [ ]:
cohort_count

## Create retention as a percentage

In [ ]:
cohort_size = cohort_count.iloc[:,0] #select all the rows : select the first column
retention = cohort_count.divide(cohort_size, axis=0) #Divide the cohort by the first column
retention.round(3) # round the retention to 3 places

### Creating cohort heatmaps

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Retention Rate')
sns.heatmap(data = retention, 
            annot = True, 
            fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.5,
            cmap = "YlGnBu")
plt.show()

## Average Quantity Sold

In [ ]:
cohort_data2 = cleaned_data.groupby(['CohortMonth', 'CohortIndex'])['Quantity'].mean().reset_index()
average_quantity  = cohort_data2.pivot_table(index = 'CohortMonth',
                                            columns = 'CohortIndex',
                                       values = 'Quantity').round(1)
average_quantity

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Average Quantity')
sns.heatmap(data = average_quantity, 
            annot = True, 
            cmap = "BuGn")
plt.show()

## Average sales

In [ ]:
#Create a new column for Total Sales 
cleaned_data['TotalSale'] = cleaned_data['Quantity'] * cleaned_data['UnitPrice']

In [ ]:
cleaned_data['TotalSale'].head()

In [ ]:
cohort_data3 = cleaned_data.groupby(['CohortMonth', 'CohortIndex'])['TotalSale'].mean().reset_index()
average_sales  = cohort_data3.pivot_table(index = 'CohortMonth',
                                           columns = 'CohortIndex',
                                      values = 'TotalSale').round(1)
average_sales

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Average Sales')
sns.heatmap(data = average_sales, 
            annot = True, 
            cmap = "Blues")
plt.show()